# RecDP LLM - sentence split

# Get started

## 1. Install pyrecdp and dependencies

In [ ]:
! DEBIAN_FRONTEND=noninteractive apt-get install -y openjdk-8-jre
! pip install pyrecdp --pre
# ! pip install 'git+https://github.com/intel/e2eAIOK.git#egg=pyrecdp&subdirectory=RecDP'

## 2. Sentence split

In [8]:
import pandas as pd
from pyrecdp.primitives.llmutils import sentence_split
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

samples = [(
        'Smithfield employs 3,700 people at its plant in Sioux Falls, '
        'South Dakota. The plant slaughters 19,500 pigs a day — 5 '
        'percent of U.S. pork.',
        'Smithfield employs 3,700 people at its plant in Sioux Falls, '
        'South Dakota.\nThe plant slaughters 19,500 pigs a day — 5 '
        'percent of U.S. pork.')]
input_dataset = spark.createDataFrame(pd.DataFrame(samples, columns=["text", "target"]))
ret_df = sentence_split(input_dataset)
ret_df.show(truncate=False)
for _, row in ret_df.toPandas().iterrows():
        assert(row["text"] == row["target"])


+--------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------+
|text                                                                                                                                        |target                                                                                                                                      |
+--------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------+
|Smithfield employs 3,700 people at its plant in Sioux Falls, South Dakota.\nThe plant slaughters 19,500 pigs a day — 5 percent of U.S. pork.|Smithf